# ICON Stencil
This stencil is based on `model/atmosphere/dycore/src/icon4py/model/atmosphere/dycore/mo_solve_nonhydro_stencil_20.py`.
It is very simple and most faithfull implementation.

A reference `NumPy` implementation can be found in `model/atmosphere/dycore/tests/stencil_tests/test_mo_velocity_advection_stencil_20.py`.


In [1]:
import os
ncpu=1
os.environ['XLA_FLAGS'] = f"--xla_force_host_platform_device_count={ncpu}"
os.environ["XLA_FLAGS"] = ("--xla_cpu_multi_thread_eigen=false "
                           "intra_op_parallelism_threads=1")
os.environ['JAX_PLATFORMS'] = "cpu"

import numpy as np
import jax
import sys
import jax.numpy as jnp
from jax import jit, grad, vmap
from jax import random

# This must be enabled when `make_jaxpr` is called, because otherwhise we get problems.
jax.config.update("jax_enable_x64", True)

import dace


In [2]:
from JaxprToSDFG import  JaxprToSDFG
t = JaxprToSDFG()

### Demo Input

### Grid Dimensions
Since we miss the GT4Py Dimension mechanism, we will make some tricks.
- `{K, Cell, Edge, Vertex}DimN` is the number of either Vertex, Cell, Edges or vertical levels.
- Valid indexes always life within the range `[0, N-1]` where `N` is the above entity.
- However, they do not denote at which position they are.



In [3]:
KDimN = 5     # Vertical levels
CellDimN = 6   # How many cells we have
EdgeDimN = 12
VertexDimN = 100


# `E2C` is the offset map that transforms _edges_ to _cells_.
#   Since a edge is adijacent to at most two cells this dimension is 2 but at least 1.
E2CDimN = 2


### Fields
These are the fields we are using as an inoput material

In [4]:
# Field defined on the edges of the grid and different in every hight level.
w_con_e = np.random.rand(EdgeDimN, KDimN).astype(np.float64)

# This is a field, which is the same for every height level.
#  It is defined for every edge but for each edge it has two values (thus `E2CDimN`) which is the result
#  of an cell to edge mapping.
c_lin_e = np.random.rand(EdgeDimN, E2CDimN).astype(np.float64)

# This is a field that defines for every cell in every height level one value.
z_w_con_c_full = np.random.rand(CellDimN, KDimN).astype(np.float64)

In [5]:
A = np.ones((1, 1, 5))


### Offset Providers
Here we define the offset providors, i.e. the connectivity.
It is important that they have to be read in a certain way.

In [6]:
# This is the level mask, the `+ 1` is taken from the test but it makes sense, at least a bit.
#  For `levelmask[Koff]` use `levelmask[:-1]`
#  and for `levelmask[Koff[+1]]` use `levelmask[1:]`.
levelmask = (np.random.rand(KDimN + 1) < 0.5).astype(np.bool_)

# As stated above, each edge is either adijacent to 1 cell (at the boundary) or 2 (in the inner part)
#  This table describes which cell are adijacent to a certain edge, in case it is undefined we have `-1`.  
edge_to_cell_table = np.array([
    [0, -1], # edge 0 (neighbours: cell 0)
    [2, -1], # edge 1
    [2, -1], # edge 2
    [3, -1], # edge 3
    [4, -1], # edge 4
    [5, -1], # edge 5
    [0, 5],  # edge 6 (neighbours: cell 0, cell 5)
    [0, 1],  # edge 7
    [1, 2],  # edge 8
    [1, 3],  # edge 9
    [3, 4],  # edge 10
    [4, 5]   # edge 11
]).astype(np.int32)

# We have a triangular grid, this means a cell is bounded by three edges.
#  In unstructured it could be, we have many geometrical forms, in this case
#  we would allocate the matrix such that it fits the largest one and fill up with `-1` if we do not need them.
cell_to_edge_table = np.array([
    [0, 6, 7],   # cell 0 (neighbors: edge 0, edge 6, edge 7)
    [7, 8, 9],   # cell 1
    [1, 2, 8],   # cell 2
    [3, 9, 10],  # cell 3
    [4, 10, 11], # cell 4
    [5, 6, 11],  # cell 5
]).astype(np.int32)


assert edge_to_cell_table.shape[0] == EdgeDimN
assert cell_to_edge_table.shape[0] == CellDimN


In [7]:
# Note that this is only the first part of the stencil.
def TestStencil(c_lin_e, w_con_e, z_w_con_c_full, levelmask, edge_to_cell_table):
    c_lin_e = jnp.expand_dims(c_lin_e, axis=-1)   # Adding a new dimensions of shape 1 at the end for propper broadcasting (taken from the test).
    
    levelmask_offset_0 = levelmask[:-1]   # This is `levelmask[Koff[0]]`, i.e. the normal field
    levelmask_offset_1 = levelmask[1:]    # This is `levelmask[Koff[1]]`, i.e. the field at the top.

    (EdgeDimN, E2CDimN, _) = c_lin_e.shape
    (CellDimN, KDimN)      = z_w_con_c_full.shape

    if(False):
        # This is more general but it requieres that a `scatter` operation is present.
        z_w_con_c_full_b = jnp.zeros((EdgeDimN, E2CDimN, KDimN))
        z_w_con_c_full_b = z_w_con_c_full_b.at[:, 0, :].set(z_w_con_c_full[edge_to_cell_table[:, 0], :]) 
        z_w_con_c_full_b = z_w_con_c_full_b.at[:, 1, :].set(z_w_con_c_full[edge_to_cell_table[:, 1], :])  # The `-1` in the `edge_to_cell_table` will be resolved as some bullshit values, that will be removed further down in the where.
    else:
        z_w_0_ = z_w_con_c_full[edge_to_cell_table[:, 0], :]
        z_w_1_ = z_w_con_c_full[edge_to_cell_table[:, 1], :]
        z_w_con_c_full_0 = jnp.expand_dims(z_w_0_, axis=1)
        z_w_con_c_full_1 = jnp.expand_dims(z_w_1_, axis=1)
        z_w_con_c_full_b = jnp.concatenate((z_w_con_c_full_0, z_w_con_c_full_1), axis=1)
    #
    MaskZ = jnp.expand_dims(edge_to_cell_table != -1, axis=-1)
    Z = jnp.where(
                MaskZ, #(grid.connectivities[E2CDim] != -1)[:, :, np.newaxis],
                c_lin_e * z_w_con_c_full_b,    #z_w_con_c_full[grid.connectivities[E2CDim]],
                0,
    )
    Y = jnp.sum(Z, axis=1)
    Mask = (levelmask_offset_0) | (levelmask_offset_1)
    w_con_e = jnp.where(Mask, Y, w_con_e)
    return w_con_e
#

In [8]:
with jax.disable_jit(disable=True):
    TestStencil_jaxpr = jax.make_jaxpr(TestStencil)(c_lin_e, w_con_e, z_w_con_c_full, levelmask, edge_to_cell_table)
print(TestStencil_jaxpr)

{ lambda ; a:f64[12,2] b:f64[12,5] c:f64[6,5] d:bool[6] e:i32[12,2]. let
    f:f64[12,2,1] = broadcast_in_dim[
      broadcast_dimensions=(0, 1)
      shape=(12, 2, 1)
    ] a
    g:bool[5] = slice[limit_indices=(5,) start_indices=(0,) strides=None] d
    h:bool[5] = slice[limit_indices=(6,) start_indices=(1,) strides=None] d
    i:i32[12,1] = slice[limit_indices=(12, 1) start_indices=(0, 0) strides=None] e
    j:i32[12] = squeeze[dimensions=(1,)] i
    k:bool[12] = lt j 0
    l:i32[12] = add j 6
    m:i32[12] = select_n k j l
    n:i32[12,1] = broadcast_in_dim[broadcast_dimensions=(0,) shape=(12, 1)] m
    o:f64[12,5] = gather[
      dimension_numbers=GatherDimensionNumbers(offset_dims=(1,), collapsed_slice_dims=(0,), start_index_map=(0,))
      fill_value=None
      indices_are_sorted=False
      mode=GatherScatterMode.PROMISE_IN_BOUNDS
      slice_sizes=(1, 5)
      unique_indices=False
    ] c n
    p:i32[12,1] = slice[limit_indices=(12, 2) start_indices=(0, 1) strides=None] e
    

In [12]:
TestStencil_sdfg = t(TestStencil_jaxpr, simplify=True, auto_opt=3)

/home/quint_essent/git/1_CSCS/jax_demo/dace/sdfg/validation.py:438: UserWarning: WARNING: Use of uninitialized transient "o" in state broadcast_in_dim_w__140293844335584
  warnings.warn('WARNING: Use of uninitialized transient "%s" in state %s' %
/home/quint_essent/git/1_CSCS/jax_demo/dace/sdfg/validation.py:438: UserWarning: WARNING: Use of uninitialized transient "v" in state broadcast_in_dim_w__140293844335584
  warnings.warn('WARNING: Use of uninitialized transient "%s" in state %s' %


In [13]:
resExp = TestStencil(c_lin_e, w_con_e, z_w_con_c_full, levelmask, edge_to_cell_table)
resDC  = TestStencil_sdfg(c_lin_e, w_con_e, z_w_con_c_full, levelmask, edge_to_cell_table)

assert np.allclose(resExp, resDC)


In [14]:
TestStencil_sdfg.view()

File saved at /tmp/tmpn1oh0q_i.sdfg.html
